## Redis Data Types
Redis has 5 primary data types:
- `STRING`: used to store strings and numbers
- `LIST`: linked list of STRING
- `SET`: unordered collection of unique STRING
- `HASH`: key value pair
- `ZSET`: sorted SET

All data stored in Redis is in key-value form. Value can be either of the 5 types described above

## STRING
The String data type is used to store String, Integer and Floating point numbers. The basic operations associated with STRING are `GET`,`SET` and `DEL`.

```bash
$ SET greeting Hello
OK
$ GET greeting
Hello
```

We can also decide to not update a key if it already exists using the `NX` option:

```bash
$ SET greeting Hi NX
(nil)
```

Keys can also be assigned a TTL using `EX <seconds>` option:

```bash
$ SET short-lived "Key with TTL" EX 5
OK
$ EXISTS short-lived
(integer) 0
```

Since the same STRING type is used to represent Integers and Floating type data, we have number specific commands. Note that a nonexisting key or blank string is equivalent to 0. The cli commands `INCR`, `DECR`, `INCRBY`, `DECRBY`

```bash
$ SET counter 50
OK
$ INCR counter
(integer) 51
$ INCRBY counter 4
(integer) 55
$ DECR unknown
(integer) -1
```

A few traditional string operations are also supported. `APPEND`, `GETRANGE` (equivalent to substring), `SETRANGE` and more bit related functions are available.

```bash
$ APPEND greeting " World"
(integer) 11
$ GET greeting
"Hello World"
$ GETRANGE greeting 6 11
"World"
$ SETRANGE greeting 6 Earth
(integer) 11
$ GET greeting
"Hello Earth"
```

To remove keys, use `DEL` command:
```bash
$ DEL greeting
(integer) 1
```

The maximum size of a string value is 512MB. Most of the operations are $O(1)$.

### LIST
Ordered sequence of STRING. The typical operations are what we expect from lists `LPUSH/RPUSH` (add to beginning or end), `LINDEX` (get item at index), `LRANGE` (get range of items), `LPOP/RPOP` (pop element from either end), `LLEN` (get length of list). Redis list is implemented as linked list, so adding new elements is faster than accessing an ith index element. Redis Lists are implemented with linked lists because for a database system it is crucial to be able to add elements to a very long list in a very fast way. 

```bash
$ RPUSH cities "new york" "london" "paris"
(integer) 3
$ LPUSH cities athens perth
(integer) 5
$ LLEN cities
(integer) 5
```

Remove items from the list:
```bash
$ LPOP cities
"perth"
$ RPOP cities
"paris"
$ LLEN cities
(integer) 3
```

Accessing specific elements of list:
```bash
$ LINDEX cities 1
"new york"
$ LRANGE cities 1 2
1) "new york"
2) "london"
```
The `LRANGE` command includes the end index. Specifying index out of bounds does not produce an error.  If start is larger than the end of the list, an empty list is returned. If stop is larger than the actual end of the list, Redis will treat it like the last element of the list. -ve offset values are also accepted:
```bash
$ LINDEX cities -1
"london"
```

Redis list is often used as stacks:
```bash
$ LPUSH stack 5
(integer) 1
$ LPUSH stack 6
(integer) 2
$ LPOP stack
"6"
```
and queues:
```bash
$ LPUSH queue 4
(integer) 1
$ LPUSH queue 7
(integer) 2
$ RPOP queue
"4"
```

We can limit the length of list using `LTRIM`:
```bash
$ RPUSH nodes node_A node_B node_C node_D node_E
(integer) 5
$ LTRIM nodes 0 2
OK
$ LRANGE nodes 0 -1
1) "node_A"
2) "node_B"
3) "node_C"
```
`LTRIM` is also helpful if we need to store for example only last *N* elements (example below shows last 5 elements):
```bash
$ LTRIM items -5 -1
```

**Blocking Operations:** consider a producer consumer setup where the producer and consumer write to/ read from the same list. Produces pushes to the list, whereas consumer pops from the list. What happens when the list is empty? The consumer can try to repeatedly poll the list for presence of new items it can pop. However, the better way is to use the blocking `BRPOP` and `BLPOP` commands.
```bash
$ RPUSH numbers 34 23
(integer) 3
$ BRPOP numbers 5
1) "numbers"
2) "23"
$ BRPOP numbers 5
1) "numbers"
2) "34"
$ BRPOP numbers 5
(nil)
(5.01s)
```
The number specified at the end is timeout interval. Setting it to zero will cause Redis to wait potentially forever. Clients are served in an ordered way: the first client that blocked waiting for a list, is served first when an element is pushed by some other client, and so forth.

### SET
SET in Redis is unordered collection of unique strings. Basic operations include `SADD` to add an item to the set, `SREM` to remove an element from the set, `SISMEMBER` check for presence of an element in set, `SMEMBERS` to return all members of the set, `SCARD` returns size of set, etc.

```bash
$ SADD cities Oslo Newark Newark Perth
(integer) 3
$ SISMEMBER cities Perth
(integer) 1
$ SREM cities Newark
(integer) 1
$ SCARD cities
(integer) 2
$ SMEMBERS cities
1) "Oslo"
2) "Perth"
```

Since this is a set, we also have the ability to intersect, union, difference:
```bash
$ SADD cities:Australia Canberra Sydney Perth Melbourne Darwin Adelaide Brisbane
(integer) 7
$ SINTER cities cities:Australia
1) "Perth"
$ SADD cities:NewZealand Auckland Wellington Queenstown Hamilton
(integer) 4
$ SUNION cities:Australia cities:NewZealand
 1) "Canberra"
 2) "Sydney"
 3) "Perth"
 4) "Melbourne"
 5) "Darwin"
 6) "Adelaide"
 7) "Brisbane"
 8) "Auckland"
 9) "Wellington"
10) "Queenstown"
11) "Hamilton"
$ SDIFF cities cities:Australia
1) "Oslo"
```
For `SDIFF`, the order of passed keys matter.

Most of the set operations like adding, removing, checking presence, are $O(1)$. `SMEMBERS` is $O(n)$.

### HASH
Redis HASH data type lets us store group of key value pairs. Both the key and the value are strings. In a lot of ways HASH can be considered as miniature version of Redis itself. Operations: `HSET`, `HGET`, `HGETALL`, `HDEL`, etc.

```bash
$ HSET porsche:911 cylinders 6 transmission manual drive rear
(integer) 3
$ HGET porsche:911 transmission
"manual"
$ HGETALL porsche:911
1) "cylinders"
2) "6"
3) "transmission"
4) "manual"
5) "drive"
6) "rear"
$ HDEL porsche:911 drive
(integer) 1
$ HGETALL porsche:911
1) "cylinders"
2) "6"
3) "transmission"
4) "manual"
$ HLEN porsche:911
(integer) 2
```

With `HEXISTS` we can check for existance of a key:
```bash
$ HEXISTS porsche:911 year
(integer) 0
$ HEXISTS porsche:911 transmission
(integer) 1
```

With `HMGET` we can get values of multiple fields:
```bash
$ HMGET porsche:911 transmission cylinders variant
1) "manual"
2) "6"
3) (nil)
$ HSET porsche:911 cylinders 4
(integer) 0
$ HGETALL porsche:911
1) "cylinders"
2) "4"
3) "transmission"
4) "manual"
```

If we want to update one of the fields, we can use the same `HSET` command:
```bash
$ HSET porsche:911 cylinders 4
(integer) 0
$ HGETALL porsche:911
1) "cylinders"
2) "4"
3) "transmission"
4) "manual"
```
To increment numeric value, we have specialised `HINCRBY` command as well:
```bash
$ HINCRBY porsche:911 cylinders 2
(integer) 6
```

`HKEYS` and `HVALS` enumerate over keys and values of the HASH respectively:
```bash
$ HKEYS porsche:911
1) "cylinders"
2) "transmission"
$ HVALS porsche:911
1) "6"
2) "manual"
```

Operations on HASH are typically $O(1)$, except for some like `HKEYS`, `HVALS` and `HGETALL` which take $O(n)$ where $n$ is the number of fields.

### ZSET
ZSET or sorted set is a collection of unique strings (members) ordered by an associated score. When more than one string has the same score, the strings are ordered lexicographically. The elements of the sorted set are sorted as per the following rule:
- if $a$ and $b$ are two items of the sorted set with different scores, then $a\gt b$ if $a.score\gt b.score$ 

To add elements to the ZSET, we pass a score alongwith the item in each case:
```bash
$ ZADD topScores 100 Max 120 Josh 86 Mark 94 Luke
(integer) 4
$ ZADD topScores 56 Jamie
```

To get elements from the sorted set, we can use `ZRANGE` and pass the range of indices (-1 means last element). `ZRANGE` returns in order low-high score. Whereas `ZREVRANGE` order is high-low. Ranges are inclusive:
```bash
$ ZRANGE topScores 0 2
1) "Jamie"
2) "Mark"
3) "Luke"
$  ZRANGE topScores 0 -1
1) "Jamie"
2) "Mark"
3) "Luke"
4) "Max"
5) "Josh"
$ ZREVRANGE topScores 0 -1
1) "Josh"
2) "Max"
3) "Luke"
4) "Mark"
5) "Jamie"
$ ZRANGE topScores 0 -1 WITHSCORES
 1) "Jamie"
 2) "56"
 3) "Mark"
 4) "86"
 5) "Luke"
 6) "94"
 7) "Max"
 8) "100"
 9) "Josh"
10) "120"
```

We can also pass score ranges in `ZRANGEBYSCORE` and `ZCOUNT` to get count of elements lying in the score range:
```bash
$ ZRANGEBYSCORE topScores 100 inf
1) "Max"
2) "Josh"
$ ZCOUNT topScores -inf 50
(integer) 0
```

`ZRANK` and `ZREVRANK` can be used to find out where in the order a particular element lies in the sorted set:
```bash
$ ZRANK topScores Luke
(integer) 2
$ ZCARD topScores
(integer) 5
$ ZREVRANK topScores Luke
(integer) 2
```

To remove elements from the sorted set, use `ZREM` or `ZREMRANGEBYSCORE` commands:
```bash
$ ZREM topScores Jamie
(integer) 1
$ ZREMRANGEBYSCORE topScores 100 inf
(integer) 2
$ ZRANGE topScores 0 -1
1) "Mark"
2) "Luke"
```

`ZINCRBY` command operates on the score of an element:
```bash
$ ZADD topScores 90 Mark
(integer) 0
$ ZINCRBY topScores 11 Mark
"101"
$ ZRANGE topScores 0 -1 WITHSCORES
1) "Luke"
2) "94"
3) "Mark"
4) "101"
```

Most sorted set operations are $O(log(n))$, where $n$ is the number of elements.